In [1]:
import os

In [2]:
os.chdir(os.path.expanduser("~/work/nbs_pipeline"))

# Get dataset

In [3]:
#| export
verbose                       = 1  
show_plots                    = False
reset_kernel                  = False
pre_configured_case           = True 
case_id                       = 7 
frequency_factor              = 1
frequency_factor_change_alias = True 
cuda_device                   = 0

In [4]:
#| export
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from dvats.load import TSArtifact, infer_or_inject_freq
import pickle
import matplotlib
import matplotlib.pyplot as plt
from tsai.data.external import convert_tsf_to_dataframe
from tsai.utils import stack_pad
#| export
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb

import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F
from dvats.memory import gpu_memory_status

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedOutputStream size changed, may indicate binary 


Octave is ready <oct2py.core.Oct2Py object at 0x7f21bc3f4310>

















In [5]:
import dvats.config as cfg_

In [6]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="02c_encoder_moment-embedding"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity = user,
    # work-nbs is a place to log draft runs
    project=project,
    group=config.wandb_group,
    job_type=job_type,
    allow_val_change=True,
    mode=config.analysis_mode,
    config=config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous = 'never' if config.use_wandb else 'must', 
    resume=False,
    name = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

#| export
import pyarrow.feather as ft
df_train = train_artifact.to_df()
print("df_train ~", df_train.shape)
print(df_train.shape)
display(df_train.head())
#| export
if verbose > 0: print("---> Sliding window | ", config.w,  " | ", config.stride )
sw = SlidingWindow(window_len=config.w, stride=config.stride, get_y=[])
if verbose > 0: print(" Sliding window | ", config.w,  " | ", config.stride, "---> | df_train ~ ", df_train.shape )
X_train, _ = sw(df_train)
if verbose > 0: print(" sw_df_train | ", config.w,  " | ", config.stride, "--->" )
print("X_train ~", X_train.shape)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/macu/work/nbs_pipeline/02c_encoder_moment-embedding.ipynb.


artifact_name is missing in original dict | toy 
valid_artifact: None
wandb_group: None
analysis_mode: online
w: 100 -> 30
alias: PulsusParadoxus-SP02 -> toy
time_col is missing in original dict | None 
train_artifact: mi-santamaria/deepvats/PulsusParadoxus-SP02:latest -> mi-santamaria/deepvats/toy:latest
mask_stateful: True
epochs: 100
batch_size: 512 -> 32
data_cols is missing in original dict | [] 
mask_sync: False
mvp_ws: (15, 100) -> [10, 30]
data_fpath is missing in original dict | ~/data/toy.csv 
freq is missing in original dict | 1s 
r: 0.71
norm_use_by_single_batch is missing in original dict | (False,) 
valid_size: 0.2
mask_future: False
csv_config is missing in original dict | {} 
norm_use_single_batch: False
use_wandb: True
stride: 900 -> 1
norm_by_sample: False
runname: 02c_encoder_moment-embedding
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch:

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


Wandb init -->
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch: False
r: 0.71
stride: 1
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
use_wandb: True
valid_size: 0.2
w: 30
wandb_group: None
artifact_name: toy
data_cols: []
data_fpath: ~/data/toy.csv
freq: 1s
time_col: None
csv_config: {}
norm_use_by_single_batch: [False]
---> W&B Train Artifact


wandb:   1 of 1 files downloaded.  


df_train ~ (550, 3)
(550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


---> Sliding window |  30  |  1
 Sliding window |  30  |  1 ---> | df_train ~  (550, 3)
 sw_df_train |  30  |  1 --->
X_train ~ (521, 3, 30)


In [7]:
X_train.shape

(521, 3, 30)

# Siguiendo el tutorial 
> (luego quito pasos que parecen extraños)

In [8]:
server_path = "Melady/TEMPO"
server_dir = server_path +" etth2_336_96_checkpoint.pth"
local_path = "./TEMPO/"
local_dir = local_path + "TEMPO_checkpoints"

In [9]:
!huggingface-cli download {server_dir} --local-dir {local_dir}

etth2_336_96_checkpoint.pth: 100%|███████████| 293M/293M [00:59<00:00, 4.94MB/s]
Download complete. Moving file to TEMPO/TEMPO_checkpoints/etth2_336_96_checkpoint.pth
TEMPO/TEMPO_checkpoints/etth2_336_96_checkpoint.pth


In [10]:
import subprocess

# Define the command as a list of arguments
command = [
    "huggingface-cli", "download",
    server_path, "stl_forfoundation/ETTh2/seasonal.pk",
    "--local-dir", local_path
]

# Execute the command
result = subprocess.run(command, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Download successful")
else:
    print("Download failed")
    print("Error message:", result.stderr)

Download successful


In [11]:
from huggingface_hub import snapshot_download
# Define the model ID and the local directory
repo_id = server_path
local_dir = local_path

# Download the folder
snapshot_download(repo_id, local_dir=local_dir, revision="main", allow_patterns="all_six_datasets/*")

print(f"Downloaded to {local_dir}")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

ETTm1.csv:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

exchange_rate.csv:   0%|          | 0.00/638k [00:00<?, ?B/s]

metr-la.csv:   0%|          | 0.00/72.8M [00:00<?, ?B/s]

ETTm2.csv:   0%|          | 0.00/9.68M [00:00<?, ?B/s]

...

ETTh2.csv:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

pems-bay.csv:   0%|          | 0.00/85.7M [00:00<?, ?B/s]

traffic.csv:   0%|          | 0.00/136M [00:00<?, ?B/s]

weather.csv:   0%|          | 0.00/7.24M [00:00<?, ?B/s]

Downloaded to ./TEMPO/


In [12]:
!mv {local_path + "all_six_datasets/"} {local_path + "dataset/"}

## Import requires packages and baseline models

In [13]:
#!mkdir content
#!mv TEMPO content/TEMPO

In [14]:
#! git clone https://github.com/DC-research/TEMPO.git

In [15]:
#ls TEMPO

In [16]:
#!pip install -r ./TEMPO/requirements.txt --no-deps

In [ ]:
#! curl -sSL https://install.python-poetry.org | python3 -
#! cd TEMPO
#! poetry init 
#! poetry add $(cat requirements.txt)
#! find ./TEMPO -type d -exec sh -c 'for dir; do [ ! -e "$dir/__init__.py" ] && touch "$dir/__init__.py"; done' sh {} +
#! poetry new --name TEMPO --src --dependency numpy TEMPO_project
"""
mv requirements.txt _requirements.txt
grep -v '^--' _requirements.txt > requirements.txt
poetry init --name TEMPO --description "TEMPO" --author "Author Name <email@example.com>" --license "MIT" --no-interaction
cat requirements.txt | xargs poetry add --no-interaction
"""

In [17]:
#! touch ./data_provider/__init__.py
#! touch ./layers/__init__.py
#path = "models"
#path =
#! touch ./{path}/__init__.py

In [18]:
ls TEMPO/utils

__init__.py  __pycache__/  metrics.py  rev_in.py  timefeatures.py  tools.py


Para que funcione, hay que Añadir los ``__init__.py'' y hacer las importaciones relativas

In [19]:
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.expanduser("~/work/nbs_pipeline/TEMPO")), '../..')))

In [20]:
from TEMPO.data_provider.data_factory import data_provider
from TEMPO.utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from torch.utils.data import Subset
from tqdm import tqdm
from nbs_pipeline.TEMPO.models.PatchTST import PatchTST
from nbs_pipeline.TEMPO.models.GPT4TS import GPT4TS
from nbs_pipeline.TEMPO.models.DLinear import DLinear
from nbs_pipeline.TEMPO.models.TEMPO import TEMPO
# from models.T5 import T54TS
from nbs_pipeline.TEMPO.models.ETSformer import ETSformer


import numpy as np
import torch

ModuleNotFoundError: No module named 'data_provider'

In [ ]:
mport torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf